In [11]:
import torch
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from datasets import Dataset
import logging
import shutil
import os

# Enable logging for monitoring
logging.basicConfig(level=logging.INFO)

# Paths to dataset
train_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Train.xlsx"
test_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Test.xlsx"

# 🔹 Define valid severity levels


train_df = pd.read_excel(train_path)
test_df = pd.read_excel(test_path)

# 🔹 Combine and select only necessary columns
all_data = pd.concat([train_df, test_df], ignore_index=True)[['q_body', 'severity', 'category']]

# 🔹 Filter rows by valid category only
valid_categories = [
    "امراض نسائية",
    "امراض العضلات والعظام و المفاصل",
    "امراض الجهاز الهضمي",
    "الامراض الجنسية",
    "طب الاسنان",
    "امراض القلب و الشرايين",
    "امراض العيون",
    "انف اذن وحنجرة",
    "جراحة تجميل",
    "امراض الدم"
]
all_data = all_data[all_data["category"].isin(valid_categories)].reset_index(drop=True)

# 🔹 Define valid severity levels
valid_severity = ["حرج", "غير حرج"]
all_data = all_data[all_data["severity"].isin(valid_severity)].reset_index(drop=True)

# 🔹 Shuffle and stratified split based on severity (not category)
all_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)
train_df, test_df = train_test_split(
    all_data[['q_body', 'severity']],  # keep only relevant columns
    test_size=0.2,
    random_state=42,
    stratify=all_data['severity']
)

# 🔹 Display distributions (optional)
print("\n🔹 Training Severity Distribution:")
print(train_df["severity"].value_counts())
print("\n🔹 Test Severity Distribution:")
print(test_df["severity"].value_counts())


# 🔹 Map severity levels to numerical labels
severity_mapping = {sev: i for i, sev in enumerate(valid_severity)}
train_df['label'] = train_df['severity'].map(severity_mapping)
test_df['label'] = test_df['severity'].map(severity_mapping)

# 🔹 Model and Tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 🔹 Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples["q_body"], padding="max_length", truncation=True, max_length=128)

# 🔹 Convert Data to Hugging Face Dataset Format
train_dataset = Dataset.from_pandas(train_df[['q_body', 'label']])
test_dataset = Dataset.from_pandas(test_df[['q_body', 'label']])

# Apply Tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 🔹 Load Base Model
base_model = AutoModel.from_pretrained(model_name)

# 🔹 Define Custom Model Class
class CustomModel(torch.nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomModel, self).__init__()  # Corrected line
        self.base_model = base_model
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(base_model.config.hidden_size, num_labels)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}


# 🔹 Initialize Model
model = CustomModel(base_model, num_labels=len(valid_severity))

# 🔹 Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=96,
    per_device_eval_batch_size=96,
    num_train_epochs=10,
    weight_decay=0.01,
    learning_rate=3e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# 🔹 Compute Metrics Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# 🔹 Trainer Initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 🔹 Start Training
print("\n🔹 Starting Training...")
trainer.train()

# 🔹 Evaluate Model on Test Dataset
test_metrics = trainer.evaluate(test_dataset)
print("\n🔹 Test Metrics:", test_metrics)

# 🔹 Get Predictions
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# 🔹 Print Evaluation Metrics
print("\n🔹 Confusion Matrix:")
print(confusion_matrix(labels, preds))

print("\n🔹 Classification Report:")
print(classification_report(labels, preds))

# 🔹 Save Model & Tokenizer



🔹 Training Severity Distribution:
severity
حرج        120301
غير حرج     38842
Name: count, dtype: int64

🔹 Test Severity Distribution:
severity
حرج        30075
غير حرج     9711
Name: count, dtype: int64


Map:   0%|          | 0/159143 [00:00<?, ? examples/s]

Map:   0%|          | 0/39786 [00:00<?, ? examples/s]

/tmp/ipykernel_35/1033345189.py:142: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



🔹 Starting Training...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.394600,0.383874,0.819082,0.840206,0.819082,0.825797
2,0.339200,0.359283,0.835017,0.849278,0.835017,0.839874
3,0.261000,0.548598,0.773714,0.853263,0.773714,0.788929
4,0.214200,0.413496,0.830694,0.860213,0.830694,0.838506
5,0.125800,0.441015,0.846831,0.860587,0.846831,0.851358
6,0.122900,0.472137,0.848640,0.860348,0.848640,0.852655
7,0.103000,0.672580,0.831172,0.857696,0.831172,0.838514
8,0.063600,0.728515,0.832981,0.859733,0.832981,0.840299
9,0.046300,0.811524,0.835947,0.859381,0.835947,0.842631
10,0.050600,0.886094,0.835922,0.859563,0.835922,0.842642



🔹 Test Metrics: {'eval_loss': 0.4721367359161377, 'eval_accuracy': 0.8486402252048459, 'eval_precision': 0.8603478544159187, 'eval_recall': 0.8486402252048459, 'eval_f1': 0.8526550288243139, 'eval_runtime': 145.7906, 'eval_samples_per_second': 272.898, 'eval_steps_per_second': 2.847, 'epoch': 10.0}

🔹 Confusion Matrix:
[[26182  3893]
 [ 2129  7582]]

🔹 Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.87      0.90     30075
           1       0.66      0.78      0.72      9711

    accuracy                           0.85     39786
   macro avg       0.79      0.83      0.81     39786
weighted avg       0.86      0.85      0.85     39786



In [12]:
def save_complete_model(model, tokenizer, severity_mapping, save_path):
    model.base_model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    classifier_state = {
        'classifier_state': model.classifier.state_dict(),
        'num_labels': model.classifier.out_features
    }
    torch.save(classifier_state, f"{save_path}/classifier_state.pt")
    with open(f"{save_path}/severity_mapping.pkl", "wb") as f:
        pickle.dump(severity_mapping, f)

# Save the model
save_complete_model(trainer.model, tokenizer, severity_mapping, "bert-base-multilingual-cased-Severity")

# 🔹 Load the Model for Inference
def load_complete_model(model_path):
    base_model = AutoModel.from_pretrained(model_path)
    classifier_state = torch.load(f"{model_path}/classifier_state.pt", map_location=torch.device('cpu'))
    model = CustomModel(base_model, classifier_state['num_labels'])
    model.classifier.load_state_dict(classifier_state['classifier_state'])
    model.eval()
    return model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased-Severity")
model2 = load_complete_model("bert-base-multilingual-cased-Severity")
print("✅ Model Loaded Successfully!")

# 🔹 Function for Severity Predictions
def predict_severity(text, model, tokenizer, severity_mapping):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    logits = outputs["logits"]
    predicted_label = torch.argmax(logits, dim=-1).item()
    severity_mapping_reverse = {v: k for k, v in severity_mapping.items()}
    return severity_mapping_reverse[predicted_label]

# 🔹 Sample Severity Predictions
test_samples = ["الاعراض شديده", "تحتاج الى راحه فقط", "حاله متوسطه", "نزيف حاد"]
for text in test_samples:
    predicted_severity = predict_severity(text, model2, tokenizer, severity_mapping)
    print(f"\n🔹 Input: {text}")
    print(f"Predicted Severity: {predicted_severity}")

# 🔹 Zip Model for Download
shutil.make_archive("bert-base-multilingual-cased-Severity", 'zip', "bert-base-multilingual-cased-Severity")
print("\n✅ Model Saved & Zipped for Download!")

# Provide a download link
from IPython.display import FileLink
FileLink(r'bert-base-multilingual-cased-Severity.zip')

✅ Model Loaded Successfully!

🔹 Input: الاعراض شديده
Predicted Severity: حرج

🔹 Input: تحتاج الى راحه فقط
Predicted Severity: حرج

🔹 Input: حاله متوسطه
Predicted Severity: حرج

🔹 Input: نزيف حاد
Predicted Severity: حرج

✅ Model Saved & Zipped for Download!


/kaggle/working/bert-base-multilingual-cased-Severity.zip

In [1]:
import wandb
wandb.login(key="72d0e227429bd347553a5563b7396b82cb04a364")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rokaia-emad (rokaia-emad-modern-sciences-and-arts-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split  # <-- Add this line
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from datasets import Dataset
import logging
import shutil
import os


# Enable logging for monitoring
logging.basicConfig(level=logging.INFO)

# Paths to dataset
train_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Train.xlsx"
test_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Test.xlsx"

# 🔹 Define valid severity levels


train_df = pd.read_excel(train_path)
test_df = pd.read_excel(test_path)

# 🔹 Combine and select only necessary columns
all_data = pd.concat([train_df, test_df], ignore_index=True)[['q_body', 'severity', 'category']]

# 🔹 Filter rows by valid category only
valid_categories = [
    "امراض نسائية",
    "امراض العضلات والعظام و المفاصل",
    "امراض الجهاز الهضمي",
    "الامراض الجنسية",
    "طب الاسنان",
    "امراض القلب و الشرايين",
    "امراض العيون",
    "انف اذن وحنجرة",
    "جراحة تجميل",
    "امراض الدم"
]
all_data = all_data[all_data["category"].isin(valid_categories)].reset_index(drop=True)

# 🔹 Define valid severity levels
valid_severity = ["حرج", "غير حرج"]
all_data = all_data[all_data["severity"].isin(valid_severity)].reset_index(drop=True)

# 🔹 Shuffle and stratified split based on severity (not category)
all_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)
train_df, test_df = train_test_split(
    all_data[['q_body', 'severity']],  # keep only relevant columns
    test_size=0.2,
    random_state=42,
    stratify=all_data['severity']
)

# 🔹 Display distributions (optional)
print("\n🔹 Training Severity Distribution:")
print(train_df["severity"].value_counts())
print("\n🔹 Test Severity Distribution:")
print(test_df["severity"].value_counts())


# 🔹 Map severity levels to numerical labels
severity_mapping = {sev: i for i, sev in enumerate(valid_severity)}
train_df['label'] = train_df['severity'].map(severity_mapping)
test_df['label'] = test_df['severity'].map(severity_mapping)

# 🔹 Model and Tokenizer
model_name = "aubmindlab/bert-base-arabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 🔹 Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples["q_body"], padding="max_length", truncation=True, max_length=128)

# 🔹 Convert Data to Hugging Face Dataset Format
train_dataset = Dataset.from_pandas(train_df[['q_body', 'label']])
test_dataset = Dataset.from_pandas(test_df[['q_body', 'label']])

# Apply Tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 🔹 Load Base Model
base_model = AutoModel.from_pretrained(model_name)

# 🔹 Define Custom Model Class
class CustomModel(torch.nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomModel, self).__init__()  # Corrected line
        self.base_model = base_model
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(base_model.config.hidden_size, num_labels)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}


# 🔹 Initialize Model
model = CustomModel(base_model, num_labels=len(valid_severity))

# 🔹 Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=96,
    per_device_eval_batch_size=96,
    num_train_epochs=10,
    weight_decay=0.01,
    learning_rate=3e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# 🔹 Compute Metrics Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# 🔹 Trainer Initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 🔹 Start Training
print("\n🔹 Starting Training...")
trainer.train()

# 🔹 Evaluate Model on Test Dataset
test_metrics = trainer.evaluate(test_dataset)
print("\n🔹 Test Metrics:", test_metrics)

# 🔹 Get Predictions
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# 🔹 Print Evaluation Metrics
print("\n🔹 Confusion Matrix:")
print(confusion_matrix(labels, preds))

print("\n🔹 Classification Report:")
print(classification_report(labels, preds))

# 🔹 Save Model & Tokenizer


2025-06-17 21:17:32.489364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750195052.664538      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750195052.716861      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



🔹 Training Severity Distribution:
severity
حرج        120301
غير حرج     38842
Name: count, dtype: int64

🔹 Test Severity Distribution:
severity
حرج        30075
غير حرج     9711
Name: count, dtype: int64


tokenizer_config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/717k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

/tmp/ipykernel_35/251172634.py:144: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.



🔹 Starting Training...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.366300,0.399708,0.811592,0.848478,0.811592,0.821169
2,0.305200,0.407458,0.813226,0.855082,0.813226,0.823340
3,0.208600,0.702634,0.751697,0.853418,0.751697,0.769072
4,0.140800,0.446717,0.838612,0.862798,0.838612,0.845339
5,0.074900,0.753390,0.808098,0.859763,0.808098,0.819446
6,0.061600,0.728312,0.823455,0.862409,0.823455,0.832774
7,0.025200,1.073315,0.811165,0.858920,0.811165,0.821994
8,0.019900,1.065887,0.827854,0.863709,0.827854,0.836617
9,0.014600,1.121916,0.831599,0.865139,0.831599,0.839920
10,0.011700,1.190708,0.834816,0.865688,0.834816,0.842662



🔹 Test Metrics: {'eval_loss': 0.4467172920703888, 'eval_accuracy': 0.838611571909717, 'eval_precision': 0.8627982624296657, 'eval_recall': 0.838611571909717, 'eval_f1': 0.8453385886669512, 'eval_runtime': 147.7081, 'eval_samples_per_second': 269.356, 'eval_steps_per_second': 2.81, 'epoch': 10.0}

🔹 Confusion Matrix:
[[25313  4762]
 [ 1659  8052]]

🔹 Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.84      0.89     30075
           1       0.63      0.83      0.71      9711

    accuracy                           0.84     39786
   macro avg       0.78      0.84      0.80     39786
weighted avg       0.86      0.84      0.85     39786



In [3]:
def save_complete_model(model, tokenizer, severity_mapping, save_path):
    model.base_model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    classifier_state = {
        'classifier_state': model.classifier.state_dict(),
        'num_labels': model.classifier.out_features
    }
    torch.save(classifier_state, f"{save_path}/classifier_state.pt")
    with open(f"{save_path}/severity_mapping.pkl", "wb") as f:
        pickle.dump(severity_mapping, f)

# Save the model
save_complete_model(trainer.model, tokenizer, severity_mapping, "baubmindlab/bert-base-arabert-Severity")

# 🔹 Load the Model for Inference
def load_complete_model(model_path):
    base_model = AutoModel.from_pretrained(model_path)
    classifier_state = torch.load(f"{model_path}/classifier_state.pt", map_location=torch.device('cpu'))
    model = CustomModel(base_model, classifier_state['num_labels'])
    model.classifier.load_state_dict(classifier_state['classifier_state'])
    model.eval()
    return model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("baubmindlab/bert-base-arabert-Severity")
model2 = load_complete_model("baubmindlab/bert-base-arabert-Severity")
print("✅ Model Loaded Successfully!")

# 🔹 Function for Severity Predictions
def predict_severity(text, model, tokenizer, severity_mapping):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    logits = outputs["logits"]
    predicted_label = torch.argmax(logits, dim=-1).item()
    severity_mapping_reverse = {v: k for k, v in severity_mapping.items()}
    return severity_mapping_reverse[predicted_label]

# 🔹 Sample Severity Predictions
test_samples = ["الاعراض شديده", "تحتاج الى راحه فقط", "حاله متوسطه", "نزيف حاد"]
for text in test_samples:
    predicted_severity = predict_severity(text, model2, tokenizer, severity_mapping)
    print(f"\n🔹 Input: {text}")
    print(f"Predicted Severity: {predicted_severity}")

# 🔹 Zip Model for Download
shutil.make_archive("baubmindlab/bert-base-arabert-Severity", 'zip', "baubmindlab/bert-base-arabert-Severity")
print("\n✅ Model Saved & Zipped for Download!")

# Provide a download link
from IPython.display import FileLink
FileLink(r'baubmindlab/bert-base-arabert-Severity')

✅ Model Loaded Successfully!

🔹 Input: الاعراض شديده
Predicted Severity: حرج

🔹 Input: تحتاج الى راحه فقط
Predicted Severity: حرج

🔹 Input: حاله متوسطه
Predicted Severity: حرج

🔹 Input: نزيف حاد
Predicted Severity: حرج

✅ Model Saved & Zipped for Download!


ValueError: Cannot display a directory using FileLink. Use FileLinks to display 'baubmindlab/bert-base-arabert-Severity'.

In [4]:
FileLink(r'baubmindlab/bert-base-arabert-Severity.zip')


/kaggle/working/baubmindlab/bert-base-arabert-Severity.zip